In [2]:
from Karlo.simulation.dataset_creation import survival_probability, intrinsic_gamma, measured_gamma
from scipy import constants
from scipy.interpolate import RegularGridInterpolator
import numpy as np
import concurrent.futures
import os
import time
from functools import partial
import math

In [3]:
def create_one_training_example(spectrum_parameters, lc_parameters, Eqg,
                                spectrum_error=None, lc_error=None, E_min=10 ** 10.55, E_max=10 ** 13.7,
                                z=0.035, photon_num=2000, interpolation_grid=None, t_observation=4 * 28 * 60,
                                verbose=False, i=0):
    photon_count = 0
    tries_count = 0
    intrinsic_time = None
    kappa2 = measured_gamma.distanceContrib(z)
    E_max = min([E_max, measured_gamma.max_energy(Eqg, t_observation, E_min, kappa2)])
    if E_max <= E_min:
        return np.nan, np.nan, i
    if interpolation_grid is not None:
        opacity_interpolator = RegularGridInterpolator((interpolation_grid[0],
                                                        interpolation_grid[1]),
                                                       interpolation_grid[2])
    else:
        opacity_interpolator = None
    if verbose:
        start_time = time.time()
    try:
        while photon_count < photon_num:
            tries_count += 1
            if tries_count > 3*photon_num:
                return 0, 0, i#np.nan, np.nan, i
            if lc_error is not None:
                A1, mean1, sigma1, A2, mean2, sigma2 = np.abs(np.random.normal(list(lc_parameters), list(lc_error)))
            else:
                A1, mean1, sigma1, A2, mean2, sigma2 = lc_parameters
            if spectrum_error is not None:
                E0, alpha = np.abs(np.random.normal(list(spectrum_parameters), list(spectrum_error)))
            else:
                E0, alpha = spectrum_parameters
            if intrinsic_time is None:
                intrinsic_time = intrinsic_gamma.intrinsic_times(A1, mean1, sigma1, A2, mean2, sigma2, size=1)
                intrinsic_energy = intrinsic_gamma.intrinsic_energy(E_min, E_max, E0, alpha, size=1)
                if opacity_interpolator is None:
                    opacity = survival_probability.opacity(intrinsic_energy[0], z, Eqg)
                else:
                    opacity = opacity_interpolator((intrinsic_energy[0], Eqg))
                survival_prob = [np.exp(-opacity)]
                survived_mask = [False]
            else:
                print("tusam")
                print("parameters", A1, mean1,sigma1, A2, mean2, sigma2,)
                intrinsic_time_tmp = intrinsic_gamma.intrinsic_times(A1, mean1,
                                                                        sigma1, A2,
                                                                        mean2, sigma2,
                                                                        size=1)
                print ("time", intrinsic_time_tmp)
                intrinsic_time = np.concatenate([intrinsic_time, intrinsic_time_tmp])
                intrinsic_energy = np.concatenate([intrinsic_energy,
                                            intrinsic_gamma.intrinsic_energy(E_min, E_max,
                                                                           E0, alpha, size=1)])
                if opacity_interpolator is None:
                    opacity = survival_probability.opacity(intrinsic_energy[-1], z, Eqg)
                else:
                    opacity = opacity_interpolator((intrinsic_energy[-1], Eqg))
                survival_prob = np.concatenate([survival_prob, [np.exp(-opacity)]])
                survived_mask = np.concatenate([survived_mask, [False]])
            if t_observation > 0:
                if (np.random.random() < survival_prob[-1]) and (
                        intrinsic_time[-1] + measured_gamma.timeDelay(intrinsic_energy[-1], Eqg,
                                                                  kappa2) - measured_gamma.timeDelay(E_min, Eqg,
                                                                                                     kappa2) < t_observation):
                    survived_mask[-1] = True
                    photon_count += 1
            else:
                if np.random.random() < survival_prob[-1]:
                    survived_mask[-1] = True
                    photon_count += 1
            if verbose:
                if photon_count != 0:
                    print("\r", photon_count, "/", photon_num, "measured, ETA:",
                        round((time.time() - start_time) * (photon_num - photon_count) / photon_count, 1), "s", )#end="   ")

        assert survived_mask.sum() == photon_num
        measured_time = intrinsic_time[survived_mask] + measured_gamma.timeDelay(intrinsic_energy[survived_mask], Eqg,
                                                                             kappa2)
        measured_energy = measured_gamma.detectionEnergy(intrinsic_energy[survived_mask], z)
        sort_index = np.argsort(measured_time)
        measured_photons = np.vstack([measured_time - measured_time.min(), measured_energy]).T[sort_index]
        intrinsic_photons = np.vstack([intrinsic_time, intrinsic_energy, survival_prob]).T[sort_index]
        return measured_photons, intrinsic_photons, i
    except Exception as e:
        print(f"Exception in worker: {e}")
        return np.nan, np.nan, i

In [ ]:
import numpy as np
#import Karlo.simulation.dataset_creation as dataset_creation

test1 =[8.94252900e+12, 2.33950394e+00, 2.77742417e+01, 4.84852408e+03, 6.35956539e+02,
        1.98512118e+01, 2.68348172e+03, 6.87631652e+02, 2.99792859e+23]
test2 =[5.58642990e+12, 2.70467552e+00, 3.36315330e+01, 3.02226309e+03, 6.03053917e+02,
        8.45283242e+01, 3.41768613e+03, 8.10011238e+02, 7.09404636e+22]


opacity_grid, E_grid, Eqg_grid = np.load("./Karlo/extra/Opacity_grid_100x100.npz").values()
interpolation_grid = (E_grid, Eqg_grid, opacity_grid)
opacity_grid, E_grid, Eqg_grid = np.load("./Karlo/extra/Opacity_grid_100x100.npz").values()
interpolation_grid = (E_grid, Eqg_grid, opacity_grid)
x_measured, x_intrinsic, _ = create_one_training_example (spectrum_parameters=test1[:2],
                                          lc_parameters=test2[2:8],
                                          Eqg=test2[8], spectrum_error=(0., 0.24), lc_error=(6., 185., 301., 11., 220., 283.),
                                          interpolation_grid=interpolation_grid, verbose=True)



In [ ]:
intrinsic_time_tmp = intrinsic_gamma.intrinsic_times(36.97348000381039, 2646.740802502216, 252.9971913535279, 
                                                     97.7719096500802, 3594.565994303601, 309.76313898064205, size=1)

In [1]:
import tensorflow as tf
import numpy as np

In [257]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import time

nPhot = 2000
#nPhot = 10
path = './Codes/photonSurvivalProbability/Run2/TrainingSamples/'
dir_list = os.listdir(path)
fileList = [i for i in dir_list if 'TrainSample_N-20_z-' in i and '.txt' in i and 'Tovar' not in i]
#fileList = [i for i in dir_list if 'Tovar_TrainSample_z-' in i and '.txt' in i]
fileList.sort(key = lambda x: float(x.split('LambdaQG-')[1].split('.txt')[0]))
#fileList = ['TrainSample_N-100_z-0.034_LambdaQG-2526479177.553053.txt']
train_x = []
train_y = []
for i in fileList:
    redshift = i.split('-')[2].split('_')[0]
    LQG = float(i.split('LambdaQG-')[1].split('.txt')[0])
    print(i, redshift, LQG)
    infile = open(path + i, 'r')
    lines = infile.readlines()
    trainSample_x = []
    for line in lines:
        #print(line)
        if line[0] == '[' and len(line) > 60:
            #print(len(line))
            trainSample_y = [str(LQG)]
            trainSample_y += line.strip('[').strip(']\n').split(',')
            #train_y = np.array(line.strip('[').strip(']\n').split(','), dtype=np.float32)
        elif line[0] == '[' and len(line) < 60:
            trainSample_y += line.strip('[').strip(']\n').split(',')
            #print(type(trainSample_y))
            #print(trainSample_y)
            train_y.append(trainSample_y)
        #elif line[0] != '[' and len(trainSample_x) < nPhot:
        elif line[0] != '[':
            #print(line.split()[0] + line.split()[1])
            try:
                trainSample_x.append([line.split()[0], line.split()[1]])
            except:
                print(line)
        #elif line[0] != '[' and len(trainSample_x) == nPhot:
            if len(trainSample_x) == nPhot:
                #print(trainSample_x)
                train_x.append(trainSample_x)
                trainSample_x = []
            #trainSample_x = [[line.split()[0], line.split()[1]]]
        #elif EOF:
         #   print(trainSample_x)
          #  train_x.append(trainSample_x)
            
train_x = np.array(train_x, dtype=np.float64)
#print(train_x.shape)
#print(train_x)
train_x = train_x.reshape(train_x.shape[0], -1, order='F').T
print(train_x.shape)
#print(train_x)
train_y = np.array(train_y, dtype=np.float32).T
print(train_y.shape)
#print(train_y)

In [265]:
plt.hist(train_x[:,0][:2000], bins=100)

In [2]:
import Karlo.simulation.dataset_creation as dataset_creation

In [3]:
opacity_grid, E_grid, Eqg_grid = np.load("./Karlo/extra/Opacity_grid_100x100.npz").values()
interpolation_grid = (E_grid, Eqg_grid, opacity_grid)
x_measured, x_intrinsic, _ = dataset_creation.create_one_training_example (spectrum_parameters=[6.37540843e+12, 2.39079769e+00],
                                          lc_parameters=[8.72911505e+01, 2.22771660e+03, 1.48336327e+03, 2.42958858e+01, 4.24120223e+03, 7.05215115e+02],
                                          Eqg=5.19556821e+16, spectrum_error=(0., 0.24), lc_error=(6., 185., 301., 11., 220., 283.),
                                          interpolation_grid=interpolation_grid, verbose=True)

In [ ]:
opacity_grid, E_grid, Eqg_grid = np.load("./Karlo/extra/Opacity_grid_100x100.npz").values()
interpolation_grid = (E_grid, Eqg_grid, opacity_grid)
x_measured, x_intrinsic, _ = dataset_creation.create_one_training_example (spectrum_parameters=test1[:2],
                                          lc_parameters=test2[2:8],
                                          Eqg=test2[8], spectrum_error=(0., 0.24), lc_error=(6., 185., 301., 11., 220., 283.),
                                          interpolation_grid=interpolation_grid, verbose=True)

In [ ]:
opacity_grid, E_grid, Eqg_grid = np.load("./Karlo/extra/Opacity_grid_100x100.npz").values()
interpolation_grid = (E_grid, Eqg_grid, opacity_grid)
x_measured, x_intrinsic, _ = dataset_creation.create_one_training_example (spectrum_parameters=test2[:2],
                                          lc_parameters=test2[2:8],
                                          Eqg=test2[8], spectrum_error=(0., 0.24), lc_error=(6., 185., 301., 11., 220., 283.),
                                          interpolation_grid=interpolation_grid, verbose=True)

In [65]:
x_measured[:,0].min(), x_measured[:,0].max()

In [66]:
x_intrinsic[:,0].min(), x_intrinsic[:,0].max()

In [86]:
_ = plt.hist([x_measured[:,0], x_intrinsic[:,0]],  bins=50, range=[0, 10000])

In [2]:
import os
import numpy as np
from scipy.stats import gaussian_kde
import scipy.stats
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator


def density_color_mask(x, y, small_size):
    x = np.array(x)
    y = np.array(y)
    xy = (np.vstack([x.ravel(), y.ravel()]))
    if x.shape[0] > small_size:
        small_sample = np.random.choice(range(x.shape[0]), size=small_size)
        z = gaussian_kde(xy[:, small_sample], bw_method='silverman')(xy)
    else:
        z = gaussian_kde(xy, bw_method='silverman')(xy)
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    return x, y, z


def plot_correlation(true, prediction, axis, name="", small_size=15000, true_name="true", limit=None):
    if type(true) is tuple or type(true) is list:
        cmaps = ['Purples', 'Greens', 'Reds', 'Blues', 'Oranges', 'YlOrBr',
                 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu',
                 'PuBuGn', 'BuGn', 'YlGn']
        axis.set_xlabel(xlabel=true_name)
        axis.set_ylabel(ylabel="NN estimated")
        if limit is None:
            limit = (true[0].min(), true[0].max())
        if (type(name) is not list) and (type(name) is not tuple or len(name) == 1):
            name = [str(i) for i in range(len(true))]
        for i in range(len(true)):
            limit = (min(limit[0], min(true[i].min(), prediction[i].min())),
                     max(limit[1], max(true[i].max(), prediction[i].max())))
            t, p, z = density_color_mask(true[i], prediction[i], small_size)
            axis.scatter(np.array(t).flatten(),
                         np.array(p).flatten(),
                         c=z, label=name[i], s=1, cmap=cmaps[i])
    else:
        cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'cividis']
        axis.set_xlabel(xlabel=name + " " + true_name)
        axis.set_ylabel(ylabel=name + " NN estimated")
        if limit is None:
            limit = (min(true.min(), prediction.min()),
                    max(true.max(), prediction.max()))
        t, p, z = density_color_mask(true, prediction, small_size)
        scatter=axis.scatter(np.array(t).flatten(),
                             np.array(p).flatten(),
                             c=z, label=name, s=1, cmap=cmaps[0])
        cbar = plt.colorbar(scatter, ax=axis, pad=0.02)
        cbar.ax.set_ylabel("Points Density", rotation=270, labelpad=-15)
        cbar.set_ticks([z.min(), z.max()])
        cbar.set_ticklabels(['low', 'high'])
    axis.plot([limit[0], limit[1]], [limit[0], limit[1]], c='red', linestyle="dashed", alpha=0.9)
    lgnd = axis.legend(scatterpoints=1, loc='lower right')
    axis.set_facecolor("white")
    limit = (limit[0] - 0.05 * (limit[1] - limit[0]), limit[1] + 0.05 * (limit[1] - limit[0]))
    axis.set(xlim=limit, ylim=limit)
    for i, handle in enumerate(lgnd.legend_handles[0:-1]):
        handle.set_sizes([10.0])
        handle.set_color(matplotlib.cm.get_cmap(cmaps[i])(0.7))
    return axis

In [122]:
import os
import numpy as np
import pickle
import random
import scipy
import scipy.stats
import datetime
from scipy.stats.sampling import NumericalInversePolynomial
from scipy import constants
from scipy import integrate
from matplotlib import pyplot as plt
from scipy.integrate import quad
from scipy.optimize import curve_fit

def doubleGauss(x, A1, mu1, sigma1, A2, mu2, sigma2):
    return np.exp(-0.5*((x-mu1)/sigma1)**2)*A1 + np.exp(-0.5*((x-mu2)/sigma2)**2)*A2


def powerLaw(x, N0, alpha):
    x0 = 10.**11
    return N0 * np.power(x/x0, alpha)    


def powerLaw2(x, N0, alpha):
    x0 = 11.
    return alpha * (x - x0) + N0
    


def fitDistributions(sample):
    #fig, (ax1, ax2) = plt.subplots(2, 1,figsize=[10, 9])
    emissionTime = sample[:,0]
    numBins1 = 30
    #ax1.hist(emissionTime, density=False, bins=numBins1)
    data_entries, bins = np.histogram(emissionTime, bins=numBins1)
    binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(numBins1)])
    popt1, pcov1 = curve_fit(doubleGauss, xdata=binscenters, ydata=data_entries, 
                             p0=[100., 2000., 2000., 50., 6000., 1000.])
    emissionEnergy = sample[:,1]
    numBins2 = 20
    logbins = np.logspace(np.log10(np.min(emissionEnergy)),np.log10(np.max(emissionEnergy)), numBins2+1)
    data_entries, bins = np.histogram(emissionEnergy, bins=logbins)
    binscenters = np.array([np.sqrt(logbins[i] * logbins[i+1]) for i in range(len(logbins)-1)])
    popt2, pcov2 = curve_fit(powerLaw, xdata=binscenters, ydata=data_entries, p0=[100., -2.], bounds=((0., -10.), (np.inf, 0.)))
    return list(popt1), list(popt2)

In [124]:
lc, spect = fitDistributions(x[120])

In [118]:
y[120, :]

In [127]:
plt.hist(x[120, :, 0], bins=30)
plt.plot(np.linspace(3600, 10), doubleGauss(np.linspace(3600, 10), lc[0],lc[1],lc[2],lc[3],lc[4],lc[5]))

In [131]:
print(lc[0], y[120, 2])
print(lc[1], y[120, 3])
print(lc[2], y[120, 4])
print(lc[3], y[120, 5])
print(lc[4], y[120, 6])
print(lc[5], y[120, 7])

In [100]:
x, y = np.load("./Karlo/extra/trainset_p2000n1000_0.npz").values()

In [224]:
fitDistributions(x[0])

In [234]:
plt.hist(np.log10(x[0,:,0]), bins=100)

In [229]:
x[0,:,0].shape

In [181]:
def liv_ann(arhitecture, x_shape):
    input_x = tf.keras.layers.Input(shape=x_shape[1:], name='input')
    x = tf.keras.layers.Flatten(name="flatten")(input_x)
    x = tf.keras.layers.BatchNormalization(name="Normalization")(x)
    assert len(arhitecture["units"]) == len(arhitecture["activations"])
    for i in range(len(arhitecture["units"])-1):
        x = tf.keras.layers.Dense(units=arhitecture["units"][i], activation=arhitecture["activations"][i],
                                  name="dense" + str(i))(x)
        x = tf.keras.layers.BatchNormalization(name="Normalization"+str(i))(x)
    output = []
    for i in range(arhitecture["units"][-1]):
        output.append(tf.keras.layers.Dense(units=1,
                                            activation=arhitecture["activations"][-1],
                                            name="output" + str(i))(x))
    model = tf.keras.Model(inputs=(input_x), outputs=output, name="ann_liv")
    return model

In [182]:
errors = [1., 1/0.24, 1/6., 1/185., 1/301., 1/11., 1/220., 1/283., 1]

In [190]:
arhitecture = {"units" : [256, 128, 16, 1],
              "activations": ["leaky_relu", "leaky_relu", "leaky_relu", "linear"]}
model = liv_ann(arhitecture, x.shape)

In [217]:
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05), loss="mse")
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01), loss="mse")
              #loss=["mse","mse","mse","mse","mse","mse","mse","mse","mse"], 
              #loss_weights=[1., 1/0.24, 1/6., 1/185., 1/301., 1/11., 1/220., 1/283., 1])

In [218]:
model.fit(x_s,y_p, batch_size=1024, validation_split=0.25, epochs=1000)

In [219]:
p = model.predict(x_s)

In [220]:
fig_correlation, ax = plt.subplots(figsize=(5, 5))
#plot_correlation(y_prepared[-1], p, ax, limit=[0,30])
plot_correlation(y_p, p, ax)

In [211]:
y_p

In [212]:
p

In [216]:
np.sqrt(np.median(np.square(y_p-p)))